# Proyecto Sprint 11: Aprendizaje automático en negocios

## Introducción 

En este proyecto, trabajamos para OilyGiant, una compañía de extracción de petróleo. Nuestro objetivo es encontrar las mejores ubicaciones para abrir 200 nuevos pozos utilizando modelos de regresión lineal.  

Para ello, analizaremos datos geológicos de tres regiones y crearemos un modelo que prediga el volumen de reservas en cada pozo. Luego, seleccionaremos los 200 pozos más rentables en cada región y estimaremos su beneficio total. Finalmente, utilizaremos bootstrapping para evaluar riesgos y asegurarnos de elegir la mejor región con máxima rentabilidad y mínimo riesgo.

In [2]:
# Librerias
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error


In [3]:
# cargar los datos
df0 = pd.read_csv('/datasets/geo_data_0.csv')
df1 = pd.read_csv('/datasets/geo_data_1.csv')
df2 = pd.read_csv('/datasets/geo_data_2.csv')

## Análisis exploratorio de datos 

In [4]:
df0.info()
print()
df1.info()
print()
df2.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Nul

In [5]:
df0.head()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [6]:
df1.head()

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [7]:
df2.head()

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


In [8]:
df0.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.500419,0.250143,2.502647,92.500000
std,0.871832,0.504433,3.248248,44.288691
min,-1.408605,-0.848218,-12.088328,0.000000
25%,-0.072580,-0.200881,0.287748,56.497507
50%,0.502360,0.250252,2.515969,91.849972
75%,1.073581,0.700646,4.715088,128.564089
max,2.362331,1.343769,16.003790,185.364347


In [9]:
df1.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,1.141296,-4.796579,2.494541,68.825000
std,8.965932,5.119872,1.703572,45.944423
min,-31.609576,-26.358598,-0.018144,0.000000
25%,-6.298551,-8.267985,1.000021,26.953261
50%,1.153055,-4.813172,2.011479,57.085625
75%,8.621015,-1.332816,3.999904,107.813044
max,29.421755,18.734063,5.019721,137.945408


In [10]:
df2.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.002023,-0.002081,2.495128,95.000000
std,1.732045,1.730417,3.473445,44.749921
min,-8.760004,-7.084020,-11.970335,0.000000
25%,-1.162288,-1.174820,0.130359,59.450441
50%,0.009424,-0.009482,2.484236,94.925613
75%,1.158535,1.163678,4.858794,130.595027
max,7.238262,7.844801,16.739402,190.029838


In [11]:
df0.duplicated().sum()

0

In [12]:
df1.duplicated().sum()

0

In [13]:
df2.duplicated().sum()

0

Nuestros datasets están limpios, es decir, no hay valores nulos, ni duplicados. 

## Entrena y prueba el modelo para cada región en geo_data_0.csv

###  Divide los datos en un conjunto de entrenamiento y un conjunto de validación en una proporción de 75:25

In [24]:
features = df0.drop(columns=['id', 'product'])  # eliminamos 'id' porque no aporta información útil
target = df0['product']

# dividir en conjunto de entrenamiento (75%) y validación (25%)
features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.25, random_state=12345
)
# entrenar el modelo de regresión lineal en la Región 0
model = LinearRegression()
model.fit(features_train, target_train)

# hacer predicciones en el conjunto de validación
predictions = model.predict(features_valid)

### Muestra el volumen medio de reservas predicho y RMSE del modelo.

In [15]:
print(f"Volumen medio de reservas predicho: {predictions.mean():.3f}")

# Calcular el error cuadrático medio (RMSE)
rmse = mean_squared_error(target_valid, predictions, squared=False)
print(f"RMSE del modelo: {rmse:.3f}")
print()
print(f"Predicciones para la Region 0", predictions)

Volumen medio de reservas predicho: 92.593
RMSE del modelo: 37.579

Predicciones para la Region 0 [ 95.89495185  77.57258261  77.89263965 ...  61.50983303 118.18039721
 118.16939229]




### Coloca todos los pasos previos en funciones, realiza y ejecuta los pasos 2.1-2.5 para los archivos 'geo_data_1.csv' y 'geo_data_2.csv'.

In [28]:
# función para entrenar y evaluar el modelo en cada región
def train_and_evaluate(region_name, df):
    print(f"\n {region_name}")

    
    # separar características (features) y variable objetivo (target)
    features = df.drop(columns=['id', 'product'])
    target = df['product']
    
    # dividir en conjunto de entrenamiento (75%) y validación (25%)
    features_train, features_valid, target_train, target_valid = train_test_split(
        features, target, test_size=0.25, random_state=12345
    )
    
    # entrenar el modelo de regresión lineal
    model = LinearRegression()
    model.fit(features_train, target_train)
    
    # predicciones en el conjunto de validación
    predictions = model.predict(features_valid)
    
    # calculo del error cuadrático medio (RMSE)
    rmse = mean_squared_error(target_valid, predictions, squared=False)
    
    # mostrar resultados
    print(f" Volumen medio de reservas predicho: {predictions.mean():.3f} barriles")
    print(f" RMSE del modelo: {rmse:.3f}\n")
    
    # devolver las predicciones y las respuestas correctas
    return predictions, target_valid

# ejecutar la función en las regiones 1 y 2
pred_1, valid_1 = train_and_evaluate("Región 1", df1)
pred_2, valid_2 = train_and_evaluate("Región 2", df2)

# guardamos los valores predichos y reales en un diccionario
predictions_dict = {
    "Región 1": {"Predicciones": pred_1, "Valores Reales": valid_1},
    "Región 2": {"Predicciones": pred_2, "Valores Reales": valid_2},
}

# Mostrar el diccionario con los resultados
print(predictions_dict)


 Región 1
 Volumen medio de reservas predicho: 68.729 barriles
 RMSE del modelo: 0.893


 Región 2
 Volumen medio de reservas predicho: 94.965 barriles
 RMSE del modelo: 40.030

{'Región 1': {'Predicciones': array([ 82.66331365,  54.43178616,  29.74875995, ..., 137.87934053,
        83.76196568,  53.95846638]), 'Valores Reales': 71751     80.859783
80493     53.906522
2655      30.132364
53233     53.906522
91141      0.000000
            ...    
12581    137.945408
18456    110.992147
73035    137.945408
63834     84.038886
43558     53.906522
Name: product, Length: 25000, dtype: float64}, 'Región 2': {'Predicciones': array([ 93.59963303,  75.10515854,  90.06680936, ...,  99.40728116,
        77.77991248, 129.03241718]), 'Valores Reales': 71751     61.212375
80493     41.850118
2655      57.776581
53233    100.053761
91141    109.897122
            ...    
12581     28.492402
18456     21.431303
73035    125.487229
63834     99.422903
43558    127.445075
Name: product, Length: 25000,

### Analiza los resultados.

**Me referiré a las regiones, de acuerdo a los números de sus datasets correspopndientes, es decir, 0, 1 y 2.**

La Región 2 parece tener la mayor cantidad de reservas predichas, pero también tiene el error más alto, lo que introduce más incertidumbre.

Mientras que la Región 1 es la más estable en términos de precisión, ya que tiene el menor error RMSE, pero es la que tiene menor volumen de reservas de barriles.

## Cálculo de ganancias

### Almacena todos los valores necesarios para los cálculos en variables separadas.

Dada la inversión de 100 millones por 200 pozos petrolíferos, de media un pozo petrolífero debe producir al menos un valor de 500,000 dólares en unidades para evitar pérdidas (esto es equivalente a 111.1 unidades). Compara esta cantidad con la cantidad media de reservas en cada región.

In [29]:
total_budget = 100_000_000  # presupuesto total 100 millones de dolares
num_wells = 200  # número de pozos
revenue_per_barrel = 4.5  # ingreso por barril en dolares
min_revenue_per_well = total_budget / num_wells  # minimo ingreso por pozo para evitar pérdidas
min_units_per_well = min_revenue_per_well / revenue_per_barrel  # minimo de barriles para evitar pérdidas

In [31]:
# volumen medio de reservas reales en cada región
mean_reserves_0 = df0['product'].mean()
mean_reserves_1 = df1['product'].mean()
mean_reserves_2 = df2['product'].mean()

# comparación con el umbral de rentabilidad
comparison = {
    "Región 0": mean_reserves_0,
    "Región 1": mean_reserves_1,
    "Región 2": mean_reserves_2,
    "Mínimo requerido para evitar pérdidas": min_units_per_well
}

# se crea un DataFrame para visualizar los resultados
comparison_df = pd.DataFrame.from_dict(comparison, orient='index', columns=['Reservas Medias (barriles)'])
comparison_df


,Reservas Medias (barriles)
Región 0,92.500000
Región 1,68.825000
Región 2,95.000000
Mínimo requerido para evitar pérdidas,111111.111111


Esto significa que cada pozo debería producir al menos 111,111.11 barriles en promedio para que la inversión no genere pérdidas.

### Conclusiones sobre cómo preparar el paso para calcular el beneficio.
Para que la inversión de 100 millones de dolares sea rentable, cada pozo debe producir al menos 111.1 barriles en promedio.
Sin embargo, los datos muestran que todas las regiones tienen un promedio menor.

Por lo que la producción promedio de un pozo no es suficiente para garantizar rentabilidad en ninguna de las regiones.

Dado que el promedio de reservas no cumple el umbral, el siguiente paso será seleccionar los 200 pozos más productivos en cada región para ver si, al enfocarnos en los mejores pozos, podemos alcanzar la rentabilidad esperada.

 Objetivo: Identificar los 200 mejores pozos en cada región con las mayores reservas predichas.
 Estrategia: Ordenar los pozos por reservas estimadas y elegir los de mayor producción.

## Escribe una función para calcular la ganancia de un conjunto de pozos de petróleo seleccionados y modela las predicciones:

### Elige los 200 pozos con los valores de predicción más altos de cada una de las 3 regiones (es decir, archivos 'csv'). Resume el volumen objetivo de reservas según dichas predicciones. Almacena las predicciones para los 200 pozos para cada una de las 3 regiones. Calcula la ganancia potencial de los 200 pozos principales por región. Presenta tus conclusiones: propón una región para el desarrollo de pozos petrolíferos y justifica tu elección.




In [32]:
# función para entrenar y hacer predicciones en cada región
def train_and_predict(df):
    """
    Entrena un modelo de regresión lineal y genera predicciones para una región.
    
    Parámetros:
    - df: DataFrame con los datos de la región.
    
    Retorna:
    - Predicciones para el conjunto de validación.
    - Valores reales del conjunto de validación.
    """
    features = df.drop(columns=['id', 'product'])
    target = df['product']
    
    # dividir en conjunto de entrenamiento y validación
    features_train, features_valid, target_train, target_valid = train_test_split(
        features, target, test_size=0.25, random_state=12345
    )
    
    # entrenar modelo de regresión lineal
    model = LinearRegression()
    model.fit(features_train, target_train)
    
    # hacer predicciones en el conjunto de validación
    predictions = model.predict(features_valid)
    
    return predictions, target_valid

# función para seleccionar los 200 pozos con mayores valores predichos
def select_top_wells(predictions, target):
    """
    Selecciona los `num_wells` pozos con mayores valores predichos.
    
    Parámetros:
    - predictions: Predicciones del modelo.
    - target: Valores reales del conjunto de validación.
    
    Retorna:
    - DataFrame con los 200 pozos seleccionados.
    """
    top_indices = predictions.argsort()[-num_wells:][::-1] #[-num_wells:] selecciona los últimos 200 índices de la lista ordenada (los 200 pozos con mayor predicción)
    return target.iloc[top_indices]                        #[::-1] Ordena en orden descendente, asegurando que los índices de los pozos con mayores predicciones estén primero.

# función para calcular la ganancia de los pozos seleccionados
def calculate_profit(selected_reserves):
    """
    Calcula la ganancia total para los pozos seleccionados.
    
    Parámetros:
    - selected_reserves: Valores reales de las reservas de los pozos seleccionados.
    
    Retorna:
    - Ganancia total esperada.
    """
    return selected_reserves.sum() * revenue_per_barrel

# entrenar modelos y obtener predicciones para cada región
pred_0, valid_0 = train_and_predict(df0)
pred_1, valid_1 = train_and_predict(df1)
pred_2, valid_2 = train_and_predict(df2)

# seleccionar los 200 pozos con mayores predicciones
top_200_0 = select_top_wells(pred_0, valid_0)
top_200_1 = select_top_wells(pred_1, valid_1)
top_200_2 = select_top_wells(pred_2, valid_2)

# calcular ganancias en cada región
profit_0 = calculate_profit(top_200_0)
profit_1 = calculate_profit(top_200_1)
profit_2 = calculate_profit(top_200_2)

# almacenar y mostrar resultados
profits = {
    "Región 0": profit_0,
    "Región 1": profit_1,
    "Región 2": profit_2
}

top_predictions = {
    "Región 0": top_200_0.values,
    "Región 1": top_200_1.values,
    "Región 2": top_200_2.values
}

# determinar la mejor región para el desarrollo
best_region = max(profits, key=profits.get)

print("Ganancias esperadas en cada región:")
print(profits)
print("\nPredicciones de los 200 mejores pozos en cada región:")
print(top_predictions)
print(f"\n La mejor región para el desarrollo es: {best_region}, ya que tiene la mayor ganancia potencial.")


Ganancias esperadas en cada región:
{'Región 0': 133208.26043139852, 'Región 1': 124150.86696681511, 'Región 2': 127103.49963599832}

Predicciones de los 200 mejores pozos en cada región:
{'Región 0': array([162.81099259, 153.63983709, 162.1534881 ,  96.89358065,
       178.87951646, 130.98568123, 173.61278737,  76.18410377,
       168.47972873, 164.60822409, 155.34482021, 155.40783108,
       118.91726054, 118.48527791, 180.36377557, 159.77978358,
       128.38051461, 182.04367351, 127.90901722, 100.81333102,
        58.42064953, 139.52359252, 156.97050342, 124.52209058,
       182.22512824, 169.85908842, 157.60560086, 147.55196181,
       147.09510109, 159.40653576, 168.64449514, 142.76343751,
       150.14541181, 122.27401519, 174.19472029, 108.94173577,
       148.54060934, 158.04433996, 121.37602505, 146.20211854,
       156.90604618, 128.39250111,  99.26611933, 143.56848341,
       178.82558367, 144.11998968, 140.13665671, 165.03092297,
       162.7699428 , 154.09969866, 163.5375

<div class="alert alert-block alert-success">  
<b>Comentario del revisor</b> <a class="tocSkip"></a><br>  
<b>Éxito</b> - Muy bien estructurada la implementación de funciones para seleccionar los mejores pozos y calcular la ganancia potencial. La estrategia de ordenar los pozos por predicciones y evaluar su rentabilidad es sólida y bien fundamentada. 
</div>

## Calcula riesgos y ganancias para cada región:
Utilizando las predicciones que almacenaste en el paso 4.2, emplea la técnica del bootstrapping con 1000 muestras para hallar la distribución de los beneficios.


In [20]:
# función para realizar bootstrapping
def bootstrap_profit(target_values):
    """
    Realiza bootstrapping con 1000 muestras para calcular la distribución de ganancias.
    
    Parámetros:
    - target_values: Valores reales de reservas de los pozos seleccionados.
    
    Retorna:
    - Lista de ganancias obtenidas en 1000 simulaciones.
    """
    bootstrap_profits = []
    for _ in range(1000):  # 1000 iteraciones de bootstrapping
        sample = target_values.sample(n=num_wells, replace=True)  # Selección con reemplazo
        bootstrap_profits.append(calculate_profit(sample))
    return bootstrap_profits

# aplicar bootstrapping en cada región
bootstrap_results = {
    "Región 0": bootstrap_profit(top_200_0),
    "Región 1": bootstrap_profit(top_200_1),
    "Región 2": bootstrap_profit(top_200_2)
}

# mostrar distribución de beneficios con bootstrapping
print("\nDistribución de beneficios con bootstrapping:")
for region, values in bootstrap_results.items():
    mean_profit = np.mean(values)
    lower_bound = np.percentile(values, 2.5)  # Percentil 2.5%
    upper_bound = np.percentile(values, 97.5) # Percentil 97.5%
    
    print(f"{region}: Media = ${mean_profit:,.2f}, Intervalo de Confianza 95% = (${lower_bound:,.2f}, ${upper_bound:,.2f})")



Distribución de beneficios con bootstrapping:
Región 0: Media = $133,212.76, Intervalo de Confianza 95% = ($130,006.42, $136,204.63)
Región 1: Media = $124,150.87, Intervalo de Confianza 95% = ($124,150.87, $124,150.87)
Región 2: Media = $127,117.89, Intervalo de Confianza 95% = ($123,589.47, $130,623.80)


La Región 0 tiene la mayor ganancia esperada, por ende la hace la mas rentable.

### Encuentra el beneficio promedio, el intervalo de confianza del 95% y el riesgo de pérdidas. La pérdida es una ganancia negativa, calcúlala como una probabilidad y luego exprésala como un porcentaje.

In [33]:
# función para realizar bootstrapping y calcular el riesgo de pérdidas
def bootstrap_profit(target_values, num_samples=1000):
    bootstrap_profits = []
    
    for _ in range(num_samples):
        sample = target_values.sample(n=num_wells, replace=True)  # selección con reemplazo
        bootstrap_profits.append(calculate_profit(sample))
    
    mean_profit = np.mean(bootstrap_profits)  # beneficio promedio
    lower_bound = np.percentile(bootstrap_profits, 2.5)  # intervalo inferior (2.5%)
    upper_bound = np.percentile(bootstrap_profits, 97.5) # intervalo superior (97.5%)

    # calcular el riesgo de pérdida (probabilidad de que la ganancia sea menor que la inversión)
    loss_risk = np.mean(np.array(bootstrap_profits) < total_budget) * 100  # expresado en porcentaje

    return mean_profit, lower_bound, upper_bound, loss_risk

# aplicar bootstrapping en cada región
bootstrap_results = {
    "Región 0": bootstrap_profit(top_200_0),
    "Región 1": bootstrap_profit(top_200_1),
    "Región 2": bootstrap_profit(top_200_2)
}

# mostrar distribución de beneficios con bootstrapping
print("\n Distribución de Beneficios con Bootstrapping:")
for region, values in bootstrap_results.items():
    mean_profit, lower_bound, upper_bound, loss_risk = values
    print(f"{region}: Media = ${mean_profit:,.2f}, Intervalo 95% = (${lower_bound:,.2f}, ${upper_bound:,.2f}), Riesgo de pérdida = {loss_risk:.2f}%")



 Distribución de Beneficios con Bootstrapping:
Región 0: Media = $133,250.66, Intervalo 95% = ($130,066.35, $136,415.56), Riesgo de pérdida = 100.00%
Región 1: Media = $124,150.87, Intervalo 95% = ($124,150.87, $124,150.87), Riesgo de pérdida = 100.00%
Región 2: Media = $127,082.31, Intervalo 95% = ($123,723.61, $130,641.44), Riesgo de pérdida = 100.00%


Todas las regiones tienen un riesgo de pérdida del 100%, lo que significa que la inversión de 100 millones nunca se recupera completamente en ninguna de ellas.

### Presenta tus conclusiones: propón una región para el desarrollo de pozos petrolíferos y justifica tu elección. ¿Coincide tu elección con la elección anterior en el punto 5.1 (de acuerdo a mi indice)?


En el punto 5.1 (cuando solo analizamos ganancias sin evaluar el riesgo), la Región 0 tenía la mayor ganancia esperada, lo que la hacía la mejor opción.

Sin embargo, al incluir la evaluación de riesgos, observamos que todas las regiones presentan un riesgo del 100%, lo que cambia la perspectiva. Es decir, no se recomienda proceder con la inversión.